In [3]:
# Configure logging first - add this at the very top of your script
import logging
logging.basicConfig(level=logging.ERROR)  # Only show ERROR level and above
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
import seaborn as sns
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
#from src.utils.logger_functions import console
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
from src.utils.logger_functions import console
from src.cmos_plotter import Conduction_speed_plotter as csp
import spikeinterface.extractors as se
import pickle
import matplotlib.pyplot as plt
from matplotlib import cm
import logging
# Silence just the matplotlib.font_manager logger
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [4]:
MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/3_Processed_Data/BCM_Test'

#RECORDINGS_PATH = os.path.join(MAIN_PATH,f'Raw_Traces')
RECORDINGS_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/2_Raw_Data/Plasticity_Experiments/Raw_Data/BCM_Test/1821'
SORTER_PATH = os.path.join(MAIN_PATH, f'Sorters_old_version')
CONDUCTION_SPEEDS_PATH = os.path.join(MAIN_PATH, f'Results/Conduction_speeds')

if not os.path.exists(CONDUCTION_SPEEDS_PATH):
    os.makedirs(CONDUCTION_SPEEDS_PATH)

metadata = load_metadata_as_dataframe(file_path=MAIN_PATH)
metadata

2025-03-27 19:40:31,740 Metadata successfully loaded. 
 First 5 rows: 
                                             Filename  DIV  Chip_ID  Network_ID
0  ID1821_3_DIV45_DATE20240405_1455_spontaneous_b...   45     1821           3
1  ID1821_2_DIV45_DATE20240405_1054_spontaneous_b...   45     1821           2
2  ID1821_1_DIV45_DATE20240405_0849_spontaneous_a...   45     1821           1
3  ID1821_3_DIV45_DATE20240405_1508_spontaneous_a...   45     1821           3
4  ID1821_2_DIV45_DATE20240405_1206_spontaneous_a...   45     1821           2


,Filename,DIV,Chip_ID,Network_ID
0,ID1821_3_DIV45_DATE20240405_1455_spontaneous_b...,45,1821,3
1,ID1821_2_DIV45_DATE20240405_1054_spontaneous_b...,45,1821,2
2,ID1821_1_DIV45_DATE20240405_0849_spontaneous_a...,45,1821,1
3,ID1821_3_DIV45_DATE20240405_1508_spontaneous_a...,45,1821,3
4,ID1821_2_DIV45_DATE20240405_1206_spontaneous_a...,45,1821,2
5,ID1821_1_DIV45_DATE20240405_0836_spontaneous_b...,45,1821,1
6,ID1821_3_DIV45_DATE20240405_1313_spontaneous_a...,45,1821,3
7,ID1821_3_DIV45_DATE20240405_1259_spontaneous_b...,45,1821,3
8,ID1821_1_DIV45_DATE20240405_0807_spontaneous_b...,45,1821,1
9,ID1821_1_DIV45_DATE20240405_1016_spontaneous_a...,45,1821,1


In [32]:
results_combined = []  # Change to a list to collect results from all recordings

for idx, filename in enumerate(metadata.Filename):
    chip_id = metadata.loc[idx, 'Chip_ID']
    div = metadata.loc[idx, 'DIV']
    cell_type = 'NGN2'
    area = metadata.loc[idx, 'Network_ID']
    
    try:
        print('Calculating conduction speeds for:', filename)
        recording = se.MaxwellRecordingExtractor(os.path.join(RECORDINGS_PATH, f'{filename}'))
        template_path = os.path.join(SORTER_PATH, f'Sorter_{filename}/wf_folder_curated/')
        
        # Get sampling rate
        sampling_rate = recording.get_sampling_frequency()
        # Get probe locations
        probe_locations = recording.get_channel_locations()
        # Extract templates
        templates = csp.load_and_extract_templates(template_path)
        # Analyze conduction speeds
        results = csp.analyze_conduction_speeds(templates, probe_locations, sampling_rate)
        
        if results is not None and len(results) > 0:  # Check if results exist and not empty
            # Store recording-specific metadata for later use in visualization
            recording_metadata = {
                'chip_id': chip_id,
                'div': div,
                'cell_type': cell_type,
                'area': area,
                'sampling_rate': sampling_rate,
                'probe_locations': probe_locations,
                'templates': templates,
                'filename': filename
            }
            
            # Add metadata to each unit's results
            for result in results:
                result['chip_id'] = chip_id
                result['div'] = div
                result['cell_type'] = cell_type
                result['area'] = area
                result['filename'] = filename
                # Makes sure the speed key is consistent
                if 'speed' in result and 'speed_ms-1' not in result:
                    result['speed_ms-1'] = result.pop('speed')
                
                # Add to the combined results list
                results_combined.append(result)
            
            # Save the metadata separately for this recording
            metadata_dict = {
                'sampling_rate': sampling_rate,
                'probe_locations': probe_locations,
                'templates': templates
            }
            
            # Save individual recording metadata
            with open(os.path.join(CONDUCTION_SPEEDS_PATH, f'metadata_{filename}.pkl'), 'wb') as f:
                pickle.dump(metadata_dict, f)
                
        else:
            print(f'No conduction speeds calculated for {filename}')
            
    except Exception as e:
        print(f'Error calculating conduction speeds for {filename}: {e}')
        continue

# Save combined results
with open(os.path.join(CONDUCTION_SPEEDS_PATH, 'Conduction_speeds_all.pkl'), 'wb') as f:
    pickle.dump(results_combined, f)

Calculating conduction speeds for: ID1821_3_DIV45_DATE20240405_1455_spontaneous_before_BCMFreq_Freq80.raw.h5
Calculating conduction speeds for: ID1821_2_DIV45_DATE20240405_1054_spontaneous_before_BCMFreq_Freq20.raw.h5
Calculating conduction speeds for: ID1821_1_DIV45_DATE20240405_0849_spontaneous_after_BCMFreq_Freq80.raw.h5
Calculating conduction speeds for: ID1821_3_DIV45_DATE20240405_1508_spontaneous_after_BCMFreq_Freq80.raw.h5
Calculating conduction speeds for: ID1821_2_DIV45_DATE20240405_1206_spontaneous_after_BCMFreq_Freq10.raw.h5
Calculating conduction speeds for: ID1821_1_DIV45_DATE20240405_0836_spontaneous_before_BCMFreq_Freq80.raw.h5
Error calculating conduction speeds for ID1821_1_DIV45_DATE20240405_0836_spontaneous_before_BCMFreq_Freq80.raw.h5: [Errno 2] No such file or directory: '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/3_Processed_Data/BCM_Test/Sorters_old_version/Sorter_ID1821_1_DIV45_DATE20240405_0836_spontaneous_before_BCMFreq_F

In [31]:
results_combined = {}
for idx, filename in enumerate(metadata.Filename):
    chip_id = metadata.loc[idx, 'Chip_ID']  # Get specific row values
    div = metadata.loc[idx, 'DIV']
    cell_type = 'NGN2'
    area =  metadata.loc[idx, 'Network_ID']
    try:
        print('Calculating conduction speeds for:', filename)
        recording = se.MaxwellRecordingExtractor(os.path.join(RECORDINGS_PATH, f'{filename}'))
        template_path = os.path.join(SORTER_PATH, f'Sorter_{filename}/wf_folder_curated/')
        
        # Get sampling rate
        sampling_rate = recording.get_sampling_frequency()
        # Get probe locations
        probe_locations = recording.get_channel_locations()
        # Extract templates
        templates = csp.load_and_extract_templates(template_path)
        # Analyze conduction speeds
        results = csp.analyze_conduction_speeds(templates, probe_locations, sampling_rate)
        
        # Add metadata and rename speed column for each result
        for result in results:
            result['speed_ms-1'] = result.pop('speed')  # Rename
        
        if results is not None:
            #add key chip_id, div, cell_type
            results_combined['unit_id'] = results['unit']
            results_combined['speed_ms-1'] = results['speed_ms-1']
            results_combined['direction'] = results['direction']
            results_combined['chip_id'] = chip_id
            results_combined['div'] = div
            results_combined['cell_type'] = cell_type
            results_combined['area'] = area
            


            console.info(f'No conduction speeds calculated for {filename}')
    except Exception as e:
        console.error(f'Error calculating conduction speeds for {filename}: {e}')
        continue
# Save results
with open(os.path.join(CONDUCTION_SPEEDS_PATH, 'Conduction_speeds_all.pkl'), 'wb') as f:
    pickle.dump(results_combined, f)

Calculating conduction speeds for: ID1821_3_DIV45_DATE20240405_1455_spontaneous_before_BCMFreq_Freq80.raw.h5


ERROR:universal:Error calculating conduction speeds for ID1821_3_DIV45_DATE20240405_1455_spontaneous_before_BCMFreq_Freq80.raw.h5: list indices must be integers or slices, not str


Calculating conduction speeds for: ID1821_2_DIV45_DATE20240405_1054_spontaneous_before_BCMFreq_Freq20.raw.h5


ERROR:universal:Error calculating conduction speeds for ID1821_2_DIV45_DATE20240405_1054_spontaneous_before_BCMFreq_Freq20.raw.h5: list indices must be integers or slices, not str


Calculating conduction speeds for: ID1821_1_DIV45_DATE20240405_0849_spontaneous_after_BCMFreq_Freq80.raw.h5


KeyboardInterrupt: 

In [30]:
results_combined = pd.DataFrame(results_combined)
results_combined

,results,chip_id,div,cell_type,area
0,"{'unit': 0, 'direction': [0.027544888110955214...",1821,45,NGN2,1
1,"{'unit': 1, 'direction': [0.0917519075790948, ...",1821,45,NGN2,1
2,"{'unit': 2, 'direction': [-0.01407682669105497...",1821,45,NGN2,1
3,"{'unit': 3, 'direction': [-0.01172877883930383...",1821,45,NGN2,1
4,"{'unit': 4, 'direction': [0.017279259936150752...",1821,45,NGN2,1
...,...,...,...,...,...
192,"{'unit': 192, 'direction': [0.0866548590670352...",1821,45,NGN2,1
193,"{'unit': 193, 'direction': [0.0308755584368640...",1821,45,NGN2,1
194,"{'unit': 194, 'direction': [0.3386812243723751...",1821,45,NGN2,1
195,"{'unit': 195, 'direction': [0.1049567354123438...",1821,45,NGN2,1


In [5]:
import numpy as np
import os
conduction_speeds = np.load(os.path.join(CONDUCTION_SPEEDS_PATH, 'Conduction_speeds_all.pkl'), allow_pickle=True)

In [ ]:
conduction_speeds

1821

In [14]:

#_ = csp.visualize_unit_propagation_V2(CONDUCTION_SPEEDS_PATH, filename, results, templates, probe_locations, 
#                                sampling_rate)
#RUN THE UPPER FUNCTION FOR EACH FILENAME IN CONDUCTION SPEEDS
for i in range(len(conduction_speeds['results'])):
    filename = conduction_speeds['filename']
    results = conduction_speeds['results']
    templates = conduction_speeds['templates']
    probe_locations = conduction_speeds['probe_locations']
    sampling_rate = conduction_speeds[i]['sampling_rate']
    _ = csp.visualize_unit_propagation_V2(CONDUCTION_SPEEDS_PATH, filename, results, templates, probe_locations, 
                                sampling_rate)


KeyError: 0

In [ ]:
csp.visualize_propagation_by_recording(CONDUCTION_SPEEDS_PATH, CONDUCTION_SPEEDS_PATH)

: 

In [ ]:
# Example usage:
# df = create_summary_visualization(CONDUCTION_SPEEDS_PATH, FIGURE_PATH, group_by='div')

# Example usage:
# visualize_propagation_by_recording(CONDUCTION_SPEEDS_PATH, FIGURE_PATH)